In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

options = Options()
options.add_argument("--headless")
# 리눅스 서버같은 GUI환경이 지원안될 때 필요한 옵션
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

url = 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=American+Malt+Whiskey&custom_rating_category%5B1%5D=American+Whiskey&custom_rating_category%5B2%5D=Australian+Whisky&custom_rating_category%5B3%5D=Austrian+Whisky&custom_rating_category%5B4%5D=Belgian+Whisky&custom_rating_category%5B5%5D=Bhutanese+Whisky&custom_rating_category%5B6%5D=Blended+Scotch+Whisky&custom_rating_category%5B7%5D=Blended+American+Whiskey&custom_rating_category%5B8%5D=Blended+Belgian+Whisky&custom_rating_category%5B9%5D=Blended+French+Whisky&custom_rating_category%5B10%5D=Blended+Grain+Scotch+Whisky&custom_rating_category%5B11%5D=Blended+Irish+Whiskey&custom_rating_category%5B12%5D=Blended+Japanese+Whisky&custom_rating_category%5B13%5D=Blended+Malt+Scotch+Whisky&custom_rating_category%5B14%5D=Blended+Scotch+Whisky&custom_rating_category%5B15%5D=Bourbon%2FTennessee+Whiskey&custom_rating_category%5B16%5D=Canadian&custom_rating_category%5B17%5D=Corn+Whiskey&custom_rating_category%5B18%5D=Danish&custom_rating_category%5B19%5D=Dutch&custom_rating_category%5B20%5D=English+Blended+Whisky&custom_rating_category%5B21%5D=English+Grain+Whisky+%26+Spirit&custom_rating_category%5B22%5D=English+Whisky&custom_rating_category%5B23%5D=Flavored+Whisky+%26+Liqueurs&custom_rating_category%5B24%5D=French+Whisky&custom_rating_category%5B25%5D=German+Whisky&custom_rating_category%5B26%5D=Icelandic+Whisky+and+Malt+Spirit&custom_rating_category%5B27%5D=Indian+Whisky&custom_rating_category%5B28%5D=Irish&custom_rating_category%5B29%5D=Irish+Single+Malt&custom_rating_category%5B30%5D=Irish+Single+Pot+Still&custom_rating_category%5B31%5D=Italian&custom_rating_category%5B32%5D=Japanese+Blended+Grain&custom_rating_category%5B33%5D=Japanese+Rice+Whisky&custom_rating_category%5B34%5D=Japanese+Single+Grain&custom_rating_category%5B35%5D=Japanese+Single+Malt&custom_rating_category%5B36%5D=Japanese+Whisky&custom_rating_category%5B37%5D=Liechtenstein&custom_rating_category%5B38%5D=Light+Whiskey&custom_rating_category%5B39%5D=Miscellaneous&custom_rating_category%5B40%5D=Multinational+Whisky&custom_rating_category%5B41%5D=New+Zealand&custom_rating_category%5B42%5D=Poit%C3%ADn&custom_rating_category%5B43%5D=Polish+Whisky&custom_rating_category%5B44%5D=Romanian&custom_rating_category%5B45%5D=Rye+Whiskey&custom_rating_category%5B46%5D=Single+Grain+Belgian+Whisky&custom_rating_category%5B47%5D=Single+Grain+Irish+Whiskey&custom_rating_category%5B48%5D=Single+Grain+Scotch&custom_rating_category%5B49%5D=Single+Malt+American+Whiskey&custom_rating_category%5B50%5D=Single+Malt+Belgian+Whisky&custom_rating_category%5B51%5D=Single+Malt+English+Whisky&custom_rating_category%5B52%5D=Single+Malt+Finnish+Whisky&custom_rating_category%5B53%5D=Single+Malt+French+Whisky&custom_rating_category%5B54%5D=Single+Malt+Indian+Whisky&custom_rating_category%5B55%5D=Single+Malt+Scotch&custom_rating_category%5B56%5D=Single+Malt+Swedish+Whisky&custom_rating_category%5B57%5D=South+African+Whisky&custom_rating_category%5B58%5D=Spanish+Whisky&custom_rating_category%5B59%5D=Swiss&custom_rating_category%5B60%5D=Taiwanese+Whisky&custom_rating_category%5B61%5D=Unspecified+Single+Malt&custom_rating_category%5B62%5D=Welsh+Whisky&custom_rating_category%5B63%5D=Wheat+Whiskey&custom_rating_category%5B64%5D=White+Whiskey&custom_rating_category%5B65%5D=World+Whisky'
driver = webdriver.Chrome(options=options)

In [3]:
def crawl_data(url):
    driver.get(url)
    print('test')